# **1. Import the libraries**

In [0]:
import cv2
import os
from google.colab import files
from matplotlib import pyplot as plt
from IPython.display import Image
from IPython.display import HTML
from base64 import b64encode
import numpy as np
import glob
import imageio
from keras.models import Model, load_model
from PIL import Image, ImageEnhance
from keras.preprocessing import image
from IPython.display import Image, HTML, display
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.xception import preprocess_input


# **2. Creation of the label file "classes.txt"**

In [0]:
!printf '%s\n' 'fire' 'no_fire' 'start_fire' > classes.txt

#  **3. Load the model**

In [0]:
!wget https://github.com/EmiGenin/defis/releases/download/defi11-model/classification_final.h5

In [0]:
model_path="classification_final.h5"   # Ici, vous devez indiquer votre modèle
classes_path = "classes.txt"
     
top_n=3
model = load_model(model_path)

# **4. Load the video**

In [0]:
!wget -O video.mp4 https://github.com/ACOOLS/IA/releases/download/video/video.mp4

# **5. Creation of the paths**

In [0]:
vid_path="video.mp4" #chemin de la video
img_dir = 'video_images/'#dossier où seront enregistrées les images 

if not os.path.exists(img_dir): #vérification si le dossier existe 
    os.makedirs(img_dir) 

# **6. Verification that the video is correctly loaded**

In [0]:
#Vérification que la vidéo a bien été téléchargée 
mp4 = open('video.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

# **7. Play video and sequencing in pictures**

In [0]:
i=0;  # un compteur qui sera utilisé comme un nom d'une image
cap=cv2.VideoCapture("video.mp4") #lire la vidéo avec opencv
if (cap.isOpened()== False): #vérification si la vidéo est bien lue
  print("Error opening video stream")
fps = cap.get(cv2.CAP_PROP_FPS)
while(cap.isOpened()):  
  # Capture frame-by-frame
  ret, frame = cap.read() #récupuré image par image
  if ret == True:     
      img_name ="image_"+ str(i) +".png"
      img_path=img_dir+img_name
      cv2.imwrite(img_path,frame)
      i=i+1
  else :
      break
cap.release()

# **8. Prediction on pictures**

In [0]:

image_path="video_images"
dataset_path="video_images"  
img_dir ="new_images/"
if os.path.exists(img_dir) == False:
    os.makedirs(img_dir)
classes = []
with open(classes_path, 'r') as f:
    classes = list(map(lambda x: x.strip(), f.readlines()))


nbr_images = len(os.listdir('video_images'))



for j in range(0, nbr_images):
  current_image="video_images/image_"+str(j)+".png" 
  img = image.load_img(current_image, target_size=(299,299)) #prendre l'image i 
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = preprocess_input(x)
  # predict
  pred = model.predict(x)[0]
  result = [(classes[i], float(pred[i]) * 100.0) for i in range(len(pred))]
  result.sort(reverse=True, key=lambda x: x[1])
  img = cv2.imread(current_image)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img = cv2.rectangle(img, (200,200), (800,500), (0,0,0), 4)
  font = cv2.FONT_HERSHEY_COMPLEX 
  img_name="image_"+str(j)+".png"
  img_path=img_dir+img_name

 
  for i in range(3):
    (class_name, prob) = result[i]
    textsize = cv2.getTextSize(class_name, font, 1, 2)[0]
    textX = (img.shape[1] - textsize[0]) / 2
    textY = (img.shape[0] + textsize[1]) / 2
    if (i == 0) :
        cv2.putText(img, class_name, (int(textX)-100, int(textY)), font, 5, (255,255,255), 6, cv2.LINE_AA)
        class_name1=class_name
        cv2.imwrite(img_path,img)
        


# **9. Writing video from annotated images**

In [0]:
fileList = []
writer= None
i=0
for file in os.listdir('new_images'):
  complete_path = 'new_images/image_' +str(i) +".png" 
  fileList.append(complete_path)
  i=i+1
writer = imageio.get_writer('test.mp4', fps=fps)

for im in fileList:
    writer.append_data(imageio.imread(im))
writer.close()


### **10. Display of the final video**

In [0]:
mp4 = open('test.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)
